### MLR Model

In [3]:
from sklearn.linear_model import LinearRegression

In [4]:
from sklearn.linear_model import LinearRegression
def MLR_Model(X_train,y_train):
    mlr = LinearRegression()
    mlr.fit(X_train,y_train)
    return mlr

### Function to make ANN model

initial_weights and bias will be from MLR

In [22]:
def ANN_Model (initial_weights,bias): 
    model = Sequential()
    model.add(Input(shape = 17,))
    model.add(Dense(1, activation = 'relu', name = 'hidden'))
    model.add(Dense(1,activation = 'linear'))


    #initialize weights and bias in hidden layer
    hidden_layer = model.get_layer('hidden')
    #layer.set_weights([weights_array, bias_array])
    hidden_layer.set_weights([np.array([initial_weights]).reshape((4, 1)), np.array([bias])])


    #compile 
    model.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))
    return model



### 5-fold MLR -ANN with RMSE and R-square (whole dataset)

In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from numpy import loadtxt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [25]:
import time

In [26]:
from keras import metrics

In [27]:
whole_data = pd.read_csv("Final Yield_Weather (Mindanao).csv") 

In [28]:
whole_data.head()

,Unnamed: 0,Year,Location,Wind_Direction,Wind_Speed_(km/h),temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),surface_pressure (hPa),et0_fao_evapotranspiration (mm),...,soil_moisture_28_to_100cm (m³/m³),direct_radiation (W/m²),diffuse_radiation (W/m²),rain (mm),weathercode (wmo code),windgusts_10m (km/h),cloudcover (%),Production (mt),Area (ha),Yield (mt/ha)
0,0,1990,AGUSAN DEL NORTE,6,4.1,25.15,78.36,20.88,975.30,0.16,...,0.28,140.02,70.89,1701.0,1,9.4,100,226.68,200.0,1.133400
1,1,1991,AGUSAN DEL NORTE,8,3.1,25.06,77.99,20.73,975.55,0.16,...,0.27,138.79,73.80,1634.2,1,11.5,100,231.68,204.0,1.135686
2,2,1992,AGUSAN DEL NORTE,8,3.1,25.39,74.29,20.11,975.90,0.17,...,0.21,144.62,73.53,1433.2,1,7.9,100,219.08,199.0,1.100905
3,3,1993,AGUSAN DEL NORTE,8,3.6,24.92,79.60,20.94,975.66,0.15,...,0.28,136.18,72.02,2017.8,1,8.3,100,231.47,203.0,1.140246
4,4,1994,AGUSAN DEL NORTE,6,3.6,24.96,79.45,20.96,975.32,0.15,...,0.29,137.68,71.24,1722.0,1,8.6,100,238.09,208.0,1.144663


In [29]:
#define target and predictors 

X = whole_data[['weathercode (wmo code)', 'temperature_2m (°C)',
       'relativehumidity_2m (%)', 'dewpoint_2m (°C)', 'rain (mm)',
       'surface_pressure (hPa)', 'cloudcover (%)',
       'et0_fao_evapotranspiration (mm)', 'vapor_pressure_deficit (kPa)',
       'Wind_Direction', 'Wind_Speed_(km/h)', 'windgusts_10m (km/h)',
       'soil_temperature_28_to_100cm (°C)',
       'soil_moisture_28_to_100cm (m³/m³)', 'direct_radiation (W/m²)',
       'diffuse_radiation (W/m²)']]
y = whole_data['Yield (mt/ha)']

In [30]:
#function for rmse
from keras import backend
 
def rmse(y_true, y_pred):
 return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [31]:
from sklearn.metrics import r2_score

In [32]:
from sklearn.linear_model import LinearRegression

In [33]:
from sklearn.linear_model import LinearRegression
def MLR_Model(X_train,y_train):
    mlr = LinearRegression()
    mlr.fit(X_train,y_train)
    return mlr

In [34]:
def ANN_Model (initial_weights,bias): 
    model = Sequential()
    model.add(Input(shape = 16,)) # number of input features 
    model.add(Dense(1, activation = 'relu', name = 'hidden')) # one neuron in hidden layer, relu activation
    model.add(Dense(1,activation = 'linear')) # one neuron in output layer, linear activation


    #initialize weights and bias in hidden layer
    hidden_layer = model.get_layer('hidden')
    #layer.set_weights([weights_array, bias_array])
    hidden_layer.set_weights([np.array([initial_weights]).reshape((16, 1)), np.array([bias])])


    #compile 
    model.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),metrics = [metrics.mean_absolute_error,rmse])
    return model



In [35]:
start_time = time.time()

#initializing KFold object
kfold = KFold(n_splits=10,shuffle=True, random_state=42)
mae_scores = []
r2_scores = []

for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Predictors for train & test
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]  # Target for train, test

    # scaling fetaure sets 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    mlr_model = MLR_Model(X_train,y_train) #perform MLR
    ann_model = ANN_Model(mlr_model.coef_,mlr_model.intercept_) #Intercept and coeff of MLR will be initailzied as ANN weights
    ann_model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0) #fit ann model to data
    y_pred = ann_model.predict(X_test) #use ann model to predict test set
    mae = mean_absolute_error(y_test, y_pred) #measure mean absolute value
    print(f"MAE: {mae}")
    r2 = r2_score(y_test,y_pred) #measure r-squared
    print(f"R2: {r2}")
    print(f"Intercept: {mlr_model.intercept_}")
    print(f"Coefficients: {list(zip(mlr_model.coef_))}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

3/3 [==============================] - 0s 0s/step
MAE: 0.31769644835619837
R2: -0.04590497287826811
Intercept: 0.8001419217240268
Coefficients: [(0.05292315649536746,), (-2.780693089135207,), (-0.9321581534665366,), (2.464329741055567,), (-0.2249199225006837,), (-0.16280536417066066,), (-0.05965252594548211,), (0.14765750696174118,), (-0.1636524684973542,), (-0.03359886697392325,), (0.050391397215022754,), (0.04456413633780846,), (0.20131034028827466,), (-0.011134786886079932,), (-0.06403389684337069,), (0.1832869047469351,)]
3/3 [==============================] - 0s 8ms/step
MAE: 0.40471611756931886
R2: -0.002380495893287593
Intercept: 0.7969807551721089
Coefficients: [(0.0415030436332838,), (-2.171293253701649,), (-0.7919106893357728,), (1.8608957472005754,), (-0.19271895768520153,), (-0.1552220993248744,), (-0.054996091707731,), (0.10081036390089904,), (-0.19742324639612552,), (-0.0433874519154373,), (0.05677336764393337,), (0.02698854050199527,), (0.2541698135440218,), (-0.02054956

In [36]:
# Predictive System

input_data = (6,4.1,25.15,78.36,20.88,975.3,0.16,0.75,26.01,0.28,140.02,70.89,1701.0,1,9.4,100,226.68,200.0)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = ann_model.predict(input_data_reshaped)
print(prediction)

InvalidArgumentError: Graph execution error:

Detected at node sequential_19/hidden/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Angel\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Angel\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1053, in launch_instance

  File "C:\Users\Angel\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 737, in start

  File "C:\Users\Angel\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Program Files\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Program Files\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\Program Files\Python311\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\Angel\App